## purpose of this notebook
* look at several PSDs for each of the series types
* find some high-correlation within-group pairs and find out which frequencies contribute to the correlation


In [29]:
import get_db_data as gdd
import analyze_signal as anaz
import numpy as np
from importlib import reload
reload(gdd);

In [33]:
con = gdd.get_db_data()

In [3]:
part = con.get_participants()

In [10]:
con.participants[:8]

[(1001, 101),
 (1001, 102),
 (1001, 103),
 (1003, 108),
 (1003, 109),
 (1004, 110),
 (1004, 111),
 (1004, 112)]

In [5]:
part[:4]

[(1001, 101), (1001, 102), (1001, 103), (1003, 108)]

In [6]:
gr = {}
for p in part:
    gr[p[0]] = gr.get(p[0],[])+[p[1]]

In [35]:
grk = list(gr.keys())
grk1 = grk[::3]
bl_signals = dict(zip(grk1, list(con.get_signal_by_series_type_and_interp_type(grk1,1,4))))
i_signals = dict(zip(grk1, list(con.get_signal_by_series_type_and_interp_type(grk1,2,4))))

#i_signals[k] = list(con.get_signal_by_series_type_and_interp_type(k,2,4))